<a href="https://colab.research.google.com/github/Shikha409/Recommendation--system/blob/main/Recommendation_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recommendation system for e commerce in deep learning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Amazon Beauty Products  dataset

Source: https://www.kaggle.com/skillsmuggler/amazon-ratings

#### Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# %matplotlib inline
plt.style.use("ggplot")

import sklearn
from sklearn.decomposition import TruncatedSVD

#### Loading the dataset

In [ ]:
import pandas as pd

# Read the data
amazon_ratings = pd.read_csv('/content/drive/MyDrive/aaaaaaaaa/ratings_Beauty.csv')

# Drop missing values
amazon_ratings = amazon_ratings.dropna()

# Drop 'Timestamp' and 'Rating' columns
data = amazon_ratings.drop(columns=['Timestamp', 'Rating'])
data = data.head(100000)
# Find user IDs that appear more than once
duplicate_user_ids = data['UserId'].value_counts()[data['UserId'].value_counts() > 1].index

# Store the rows for each duplicate user ID in arrays
duplicate_users_data = {user_id: data[data['UserId'] == user_id].to_dict('records') for user_id in duplicate_user_ids}

# Aggregate product IDs for each duplicate user ID
aggregated_data = []
for user_id, records in duplicate_users_data.items():
    all_product_ids = [record['ProductId'] for record in records]
    aggregated_data.append({'UserId': user_id, 'ProductIds': all_product_ids})

# Convert the aggregated data to a DataFrame
df_aggregated = pd.DataFrame(aggregated_data)

# Save the DataFrame to a CSV file
df_aggregated.to_csv('/content/user_products.csv', index=False)

# If you want to see the first few rows of the DataFrame
print(df_aggregated.head())


           UserId                                         ProductIds
0  A281NPSIMI1C2R  [B000050B6X, B000050FET, B000052WYL, B000052Y3...
1  A3M174IC0VXOS2  [B00000JGVX, B000052YKU, B000052YMQ, B000052Z8...
2  A1Z54EM24Y40LL  [B000053037, B00005303A, B00005304H, B00005307...
3   AKMEY1BSHSDG7  [B000052WYD, B0000530LP, B0000530O6, B0000536F...
4   AK7CAW3N1XZV6  [B000052Y1Q, B000052YJX, B000052YJY, B000052YM...


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the dataset
data = pd.read_csv('/content/user_products.csv')

# Create a tokenizer for product IDs
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['ProductIds'].apply(lambda x: x.strip('[]').replace("'", "").split(', ')))

# Convert product IDs to sequences
sequences = tokenizer.texts_to_sequences(data['ProductIds'].apply(lambda x: x.strip('[]').replace("'", "").split(', ')))

# Pad sequences to ensure equal length
max_sequence_len = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')

# Create predictors and label
X = sequences[:, :-1]
y = sequences[:, -1]

# Convert labels to categorical
y = to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 50, input_length=max_sequence_len - 1))
model.add(LSTM(100))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 78, 50)            154600    
                                                                 
 lstm_1 (LSTM)               (None, 100)               60400     
                                                                 
 dense_1 (Dense)             (None, 3092)              312292    
                                                                 
Total params: 527292 (2.01 MB)
Trainable params: 527292 (2.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
151/151 [==============================] - 8s 42ms/step - loss: 7.4602 - accuracy: 0.0112 - val_loss: 7.2491 - val_accuracy: 0.0158
Epoch 2/20
151/151 [==============================] - 3s 21ms/step - loss: 6.8535 - accuracy: 0.0110 - val_loss: 7.3214 - val_accuracy: 0.0166
Epoch 3/20
151/151 [==============================] - 3s 15ms/step - loss: 6.7261 - accuracy: 0.0135 - val_loss: 7.4380 - val_accuracy: 0.0108
Epoch 4/20
151/151 [==============================] - 2s 15ms/step - loss: 6.6503 - accuracy: 0.0131 - val_loss: 7.5832 - val_accuracy: 0.0075
Epoch 5/20
151/151 [==============================] - 2s 12ms/step - loss: 6.5944 - accuracy: 0.0164 - val_loss: 7.7156 - val_accuracy: 0.0266
Epoch 6/20
151/151 [==============================] - 2s 10ms/step - loss: 6.5396 - accuracy: 0.0177 - val_loss: 7.7202 - val_accuracy: 0.0200
Epoch 7/20
151/151 [==============================] - 1s 9ms/step - loss: 6.4808 - accuracy: 0.0187 - val_loss: 7.8712 - val_accuracy: 0.0191


In [ ]:
### to predict only 1 product

# import numpy as np

# # Function to predict the next product
# def predict_next_product(model, tokenizer, current_sequence, max_sequence_len):
#     token_list = tokenizer.texts_to_sequences([current_sequence])[0]
#     token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
#     predicted_probs = model.predict(token_list, verbose=0)
#     predicted = np.argmax(predicted_probs, axis=-1)
#     return tokenizer.index_word[predicted[0]]

# # Example prediction
# sample_sequence = data['ProductIds'][0].strip('[]').replace("'", "").split(', ')
# next_product = predict_next_product(model, tokenizer, sample_sequence, max_sequence_len)
# print(f'Next product prediction: {next_product}')


### to predict 5 products ###

# Function to predict the next n products
def predict_next_n_products(model, tokenizer, current_sequence, max_sequence_len, n):
    token_list = tokenizer.texts_to_sequences([current_sequence])[0]
    predictions = []
    for _ in range(n):
        token_list_padded = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list_padded, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)
        next_product = tokenizer.index_word[predicted[0]]
        predictions.append(next_product)
        token_list.append(predicted[0])
    return predictions

# Example prediction for next 5 products
sample_sequence = data['ProductIds'][5].strip('[]').replace("'", "").split(', ')
next_products = predict_next_n_products(model, tokenizer, sample_sequence, max_sequence_len, 5)
print(f'Next 5 product predictions: {next_products}')


Next 5 product predictions: ['b0002gpeia', 'b0002ocqvk', 'b0002ocqvk', 'b0002ocqvk', 'b0002gpeia']
